Importing necessary libraries

In [ ]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
import torch
import torchvision
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms,utils
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split,Dataset
from tqdm.notebook import tqdm as tqdm
import torch.utils.data as data
import torchvision.models as models
import seaborn as sns
import torch.quantization as quantization
import warnings
import pickle
import cv2
warnings.filterwarnings('ignore')
sns.set_theme()
import torchvision.transforms as transforms
from torchvision import datasets, transforms
%matplotlib inline
plt.ion() #Turn interactive mode on.
torch.manual_seed(34)
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import time
import tensorflow as tf

In [ ]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

Dataset Generation

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d adityasaxena2/visdrone1

In [ ]:
import zipfile
import os

for file in os.listdir():
    if file.endswith(".zip"):
        zip_file = file

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
train_root_dir = '/content/train'
val_root_dir = '/content/valid'
test_root_dir = '/content/test'

Creating Train, Test, Validation directory

In [ ]:
image_list_train = sorted(os.listdir(os.path.join(train_root_dir, 'images')))
label_list_train = sorted(os.listdir(os.path.join(train_root_dir, 'labels')))

image_list_val = sorted(os.listdir(os.path.join(val_root_dir, 'images')))
label_list_val = sorted(os.listdir(os.path.join(val_root_dir, 'labels')))

image_list_test = sorted(os.listdir(os.path.join(test_root_dir, 'images')))
label_list_test = sorted(os.listdir(os.path.join(test_root_dir, 'labels')))

In [ ]:
image_list_train_new=[]
label_list_train_new=[]
start = 0
i=0
while(i<10):
  for k in range(start,start+100):
    image_list_train_new.append(image_list_train[k])
    label_list_train_new.append(label_list_train[k])
  start+= 600
  i+=1


In [ ]:
image_list_test_new=[]
label_list_test_new=[]
start = 0
i=0
while(i<10):
  for k in range(start,start+50):
    image_list_test_new.append(image_list_test[k])
    label_list_test_new.append(label_list_test[k])
  start+= 160
  i+=1


In [ ]:
print(len(image_list_train_new))
print(len(image_list_val))
# print(len(label_list_test_new))

1000
548


In [ ]:
# function to create image-label pairs
def image_label_pairs(image_list,label_list,root_dir):
  img_label_pair = {}
  for idx in range(len(image_list)):
    img_path = os.path.join(root_dir, 'images', image_list[idx])
    label_path = os.path.join(root_dir, 'labels', label_list[idx])
    img = Image.open(img_path)
    img = img.convert('RGB')
    classes = {}
    with open(label_path, 'r') as f:
      for line in f:
        l2 = line.split(',')
        label = l2[-2]
        if l2[-2] not in classes:
          classes[l2[-2]] = 1
        else:
          classes[l2[-2]] += 1
    max_value = max(classes.values())
    max_keys = [k for k, v in classes.items() if v == max_value]
    label = max_keys[0]

    img_label_pair[idx] = []
    img_label_pair[idx].append(img)
    img_label_pair[idx].append(int(label))

  return img_label_pair

In [ ]:
img_label_pairs_train = image_label_pairs(image_list_train_new,label_list_train_new,train_root_dir)

In [ ]:
img_label_pairs_val = image_label_pairs(image_list_val,label_list_val,val_root_dir)

In [ ]:
img_label_pairs_test = image_label_pairs(image_list_test_new,label_list_test_new,test_root_dir)

In [ ]:
# function to reshape the image to (224*224*3)
def reshape_img(image, target_shape):
    resized_image = cv2.resize(image, target_shape, interpolation=cv2.INTER_AREA)

    if resized_image.shape[0] < target_shape[0] or resized_image.shape[1] < target_shape[1]:
        pad_height = max(target_shape[0] - resized_image.shape[0], 0)
        pad_width = max(target_shape[1] - resized_image.shape[1], 0)
        resized_image = cv2.copyMakeBorder(
            resized_image, 0, pad_height, 0, pad_width,
            cv2.BORDER_CONSTANT, value=(0, 0, 0)
        )

    return resized_image

In [ ]:
# function to create final test, train directories
def xdata_ydata_array(img_label_pairs):
  x_data = []
  y_data = []
  for idx in range(len(img_label_pairs)):
    image = np.array(img_label_pairs[idx][0])
    image = reshape_img(image, (224, 224))
    x_data.append(image)
    y_data.append(img_label_pairs[idx][1])

  x_data = np.array(x_data)
  y_data = np.array(y_data)
  y_data = tf.convert_to_tensor(y_data)
  # y_data = tf.one_hot(y_data.astype(np.int32), depth=2)
  return x_data,y_data

In [ ]:
x_train,y_train = xdata_ydata_array(img_label_pairs_train)

In [ ]:
x_val,y_val = xdata_ydata_array(img_label_pairs_val)

In [ ]:
x_test,y_test = xdata_ydata_array(img_label_pairs_test)

## Attention Block

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

class AttentionBlock(layers.Layer):
    def __init__(self, in_channels):
        super(AttentionBlock, self).__init__()
        self.conv = layers.Conv2D(in_channels // 8, kernel_size=1)
        self.pool = layers.GlobalAveragePooling2D()
        self.fc1 = layers.Dense(in_channels)
        self.fc2 = layers.Dense(in_channels)
        self.relu = layers.ReLU()
        self.sigmoid = layers.Activation('sigmoid')

    def call(self, inputs):
        y = self.conv(inputs)
        y = self.relu(y)
        y = self.pool(y)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y)
        return inputs * tf.expand_dims(tf.expand_dims(y, 1), 1)

def AttentionCNN(flatten_shape=512*7*7, num_classes=2, in_channels=3):
    model = models.Sequential([
        layers.Conv2D(64, kernel_size=7, strides=2, padding='same', input_shape=(224, 224, in_channels)),
        AttentionBlock(64),
        layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
        AttentionBlock(128),
        layers.Conv2D(256, kernel_size=3, strides=2, padding='same'),
        AttentionBlock(256),
        layers.Conv2D(512, kernel_size=3, strides=2, padding='same'),
        AttentionBlock(512),
        layers.Conv2D(512, kernel_size=3, strides=2, padding='same'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096),
        layers.Activation('relu'),
        layers.Dense(2048),
        layers.Activation('relu'),
        layers.Dense(num_classes)
    ])
    return model

model = AttentionCNN()
model.summary()


Training the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

model = AttentionCNN()

loss_fn = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=0.001)
metrics = [SparseCategoricalAccuracy()]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
batch_size = 32
epochs = 5

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

# Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print the test accuracy
# print(f'Test accuracy: {test_accuracy}')


Epoch 1/5
32/32 [==============================] - 255s 8s/step - loss: 0.0368 - sparse_categorical_accuracy: 0.9700 - val_loss: 91.5917 - val_sparse_categorical_accuracy: 0.9982
Epoch 2/5
32/32 [==============================] - 236s 7s/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 113.8482 - val_sparse_categorical_accuracy: 0.9982
Epoch 3/5
32/32 [==============================] - 220s 7s/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 114.9559 - val_sparse_categorical_accuracy: 0.9982
Epoch 4/5
32/32 [==============================] - 221s 7s/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 115.0017 - val_sparse_categorical_accuracy: 0.9982
Epoch 5/5
32/32 [==============================] - 212s 7s/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 115.0027 - val_sparse_categorical_accuracy: 0.9982


In [ ]:
# Saving the model
import pickle

with open('attention.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Downloading the model
from google.colab import files

files.download("attention.pkl")

In [ ]:
# loading the model
import pickle

pkl_file_path = '/content/attention.pkl'
with open(pkl_file_path, 'rb') as file:
    model = pickle.load(file)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 112, 112, 64)      9472      
                                                                 
 attention_block_8 (Attentio  (None, 112, 112, 64)     5256      
 nBlock)                                                         
                                                                 
 conv2d_20 (Conv2D)          (None, 56, 56, 128)       73856     
                                                                 
 attention_block_9 (Attentio  (None, 56, 56, 128)      20752     
 nBlock)                                                         
                                                                 
 conv2d_22 (Conv2D)          (None, 28, 28, 256)       295168    
                                                                 
 attention_block_10 (Attenti  (None, 28, 28, 256)     

In [ ]:
# layer wise weight analysis
all_weights = []
for layer in model.layers:
    if hasattr(layer, 'get_weights'):
        weights = layer.get_weights()
        all_weights.append(weights)

for i, weights in enumerate(all_weights):
    print(f"Layer {i + 1} - {model.layers[i].name}")
    for j, w in enumerate(weights):
        print(f"   Weights {j + 1}: {w.shape}")

Layer 1 - conv2d_18
   Weights 1: (7, 7, 3, 64)
   Weights 2: (64,)
Layer 2 - attention_block_8
   Weights 1: (1, 1, 64, 8)
   Weights 2: (8,)
   Weights 3: (8, 64)
   Weights 4: (64,)
   Weights 5: (64, 64)
   Weights 6: (64,)
Layer 3 - conv2d_20
   Weights 1: (3, 3, 64, 128)
   Weights 2: (128,)
Layer 4 - attention_block_9
   Weights 1: (1, 1, 128, 16)
   Weights 2: (16,)
   Weights 3: (16, 128)
   Weights 4: (128,)
   Weights 5: (128, 128)
   Weights 6: (128,)
Layer 5 - conv2d_22
   Weights 1: (3, 3, 128, 256)
   Weights 2: (256,)
Layer 6 - attention_block_10
   Weights 1: (1, 1, 256, 32)
   Weights 2: (32,)
   Weights 3: (32, 256)
   Weights 4: (256,)
   Weights 5: (256, 256)
   Weights 6: (256,)
Layer 7 - conv2d_24
   Weights 1: (3, 3, 256, 512)
   Weights 2: (512,)
Layer 8 - attention_block_11
   Weights 1: (1, 1, 512, 64)
   Weights 2: (64,)
   Weights 3: (64, 512)
   Weights 4: (512,)
   Weights 5: (512, 512)
   Weights 6: (512,)
Layer 9 - conv2d_26
   Weights 1: (3, 3, 512, 51

Extracting weights from all the layers

In [ ]:
conv1_weights, conv1_biases = model.layers[0].get_weights()
at1_conv_weights, at1_conv_biases, at1_fc1_weights, at1_fc1_biases, at1_fc2_weights, at1_fc2_biases = model.layers[1].get_weights()

conv2_weights, conv2_biases = model.layers[2].get_weights()
at2_conv_weights, at2_conv_biases, at2_fc1_weights, at2_fc1_biases, at2_fc2_weights, at2_fc2_biases = model.layers[3].get_weights()

conv3_weights, conv3_biases = model.layers[4].get_weights()
at3_conv_weights, at3_conv_biases, at3_fc1_weights, at3_fc1_biases, at3_fc2_weights, at3_fc2_biases = model.layers[5].get_weights()

conv4_weights, conv4_biases = model.layers[6].get_weights()
at4_conv_weights, at4_conv_biases, at4_fc1_weights, at4_fc1_biases, at4_fc2_weights, at4_fc2_biases = model.layers[7].get_weights()

conv5_weights, conv5_biases = model.layers[8].get_weights()

dense1_weights, dense1_biases = model.layers[10].get_weights()
dense2_weights, dense2_biases = model.layers[12].get_weights()
dense3_weights, dense3_biases = model.layers[14].get_weights()

In [ ]:
# Converting biases to lists
conv1_biases_list = conv1_biases.tolist()
at1_conv_biases_list = at1_conv_biases.tolist()
at1_fc1_biases_list = at1_fc1_biases.tolist()
at1_fc2_biases_list = at1_fc2_biases.tolist()

conv2_biases_list = conv2_biases.tolist()
at2_conv_biases_list = at2_conv_biases.tolist()
at2_fc1_biases_list = at2_fc1_biases.tolist()
at2_fc2_biases_list = at2_fc2_biases.tolist()

conv3_biases_list = conv3_biases.tolist()
at3_conv_biases_list = at3_conv_biases.tolist()
at3_fc1_biases_list = at3_fc1_biases.tolist()
at3_fc2_biases_list = at3_fc2_biases.tolist()

conv4_biases_list = conv4_biases.tolist()
at4_conv_biases_list = at4_conv_biases.tolist()
at4_fc1_biases_list = at4_fc1_biases.tolist()
at4_fc2_biases_list = at4_fc2_biases.tolist()

conv5_biases_list = conv5_biases.tolist()
dense1_biases_list = dense1_biases.tolist()
dense2_biases_list = dense2_biases.tolist()
dense3_biases_list = dense3_biases.tolist()

In [ ]:
# Converting dense weights to lists
at1_fc1_weights_list = at1_fc1_weights.flatten().tolist()
at1_fc2_weights_list = at1_fc2_weights.flatten().tolist()
at2_fc1_weights_list = at2_fc1_weights.flatten().tolist()
at2_fc2_weights_list = at2_fc2_weights.flatten().tolist()
at3_fc1_weights_list = at3_fc1_weights.flatten().tolist()
at3_fc2_weights_list = at3_fc2_weights.flatten().tolist()
at4_fc1_weights_list = at4_fc1_weights.flatten().tolist()
at4_fc2_weights_list = at4_fc2_weights.flatten().tolist()

dense1_weights_list = dense1_weights.flatten().tolist()
dense2_weights_list = dense2_weights.flatten().tolist()
dense3_weights_list = dense3_weights.flatten().tolist()

In [ ]:
# flattening the weights array
new_conv_weights = []
for num_fil in range(0,64):
  for dep in range(0,512):
    for row in range(0,1):
      for col in range(0,1):
        new_conv_weights.append(at4_conv_weights[row][col][dep][num_fil])

Quantization of weights

In [ ]:
import torch

def absmax_quantize_list(weights_list):
    X = torch.tensor(weights_list, dtype=torch.float32)
    scale = 127 / torch.max(torch.abs(X))

    # Quantize
    X_quant = (scale * X).round()

    # Dequantize
    X_dequant = X_quant / scale

    quantized_list = X_quant.to(torch.int8).tolist()
    return quantized_list, X_dequant.tolist()

In [ ]:
weights_list = dense3_weights_list
# print(len(weights_list))

quantized_weights, dequantized_weights = absmax_quantize_list(weights_list)
# print(len(quantized_weights))

# saving the weights
with open('quantized_dense3_weights.txt', 'w') as f:
  f.write(str(quantized_weights))

test image

In [ ]:
# flattening the image
flattenedImage = []
for k in range(0, 3):
  for i in range(0, 224):
    for j in range(0, 224):
      flattenedImage.append(test_image[i][j][k])

In [ ]:
with open('test_image_attention.txt', 'w') as f:
  f.write(str(flattenedImage))